# LCEL

### ENV

In [37]:
# 引入依赖包，这里的 pydantic 版本为 v2
from pydantic import BaseModel, Field, model_validator
from langchain_deepseek import ChatDeepSeek
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser, StrOutputParser
from langchain.output_parsers import RetryOutputParser, OutputFixingParser
from typing import Dict
from pydantic import BaseModel, Field

# 使用 deepseek
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    api_key="sk-6b2a3015b6094e68aa2956124ad1e870",
    api_base="https://api.deepseek.com",
)

#### 1 Runnable接口

`注意:并不是所有的组件都支持所有的统一的调用方法，尤其是异步事件`

#### Pydantic 是什么

```
Pydantic 是一个 Python 数据验证和数据模型库，它的主要作用是：
1. 定义数据模型（Model），并自动验证数据类型、格式、必填字段等 
2. 自动进行类型转换（比如字符串 "123" 会自动转成 int）
3. 和 Python 类型注解（type hints）深度结合，写起来简洁
```

In [38]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    is_active: bool = True  # 默认值

user = User(id="123", name="Alice")  
print(user.id)        # 123（自动转为 int）
print(user.is_active) # True

123
True


In [39]:
# 如果数据类型不匹配或字段缺失，它会直接报错：
User(id="abc", name="Bob")
# pydantic.error_wrappers.ValidationError: value is not a valid integer

ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

```
LangChain 是一个大语言模型（LLM）应用框架，用来把 LLM 与各种工具、数据源连接起来。
LangChain 里很多数据结构（比如 Prompt 模型、Chain 参数、输出格式等）都是用 Pydantic 定义的，因为：
1. Pydantic 可以自动验证传入的参数
2. 定义清晰的数据模型方便调试、序列化、反序列化
3. 可以和 JSON、dict 轻松互转，适合 LLM 输出解析
```

In [7]:
from pydantic import BaseModel
from langchain.output_parsers import PydanticOutputParser

class Joke(BaseModel):
    setup: str
    punchline: str

parser = PydanticOutputParser(pydantic_object=Joke)

print(parser.get_format_instructions())
# 会输出提示 LLM 如何返回 Joke 格式的 JSON

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "type": "string"}, "punchline": {"title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```


### 2 LCEL

In [12]:
prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话,不要有任何解释")
chain = prompt | llm | StrOutputParser()
res = chain.invoke({"topic":"狗"})
print(res)

有一天，一只狗走进酒吧，跳到吧台上对酒保说：“你们这有速递服务吗？”  

酒保：“没有，但我们可以帮你叫外卖。”  

狗摇摇头：“不用了，我只是想确认一下——因为门口写着‘热狗速递’！”


In [ ]:
for chunk in chain.stream("小狗"):
    print(chunk, end="", flush=True)

有一天，一只小狗走进了一家酒吧。  

它跳到吧台上，对酒保说：“我要一杯啤酒，再来一份报纸。”  

酒保惊讶地问：“哇，你会说话？还会看报纸？”  

小狗耸耸肩：“报纸是垫啤酒杯的，我可不想弄湿爪子。”

In [30]:
async def stream_response():
    async for chunk in chain.astream("小狗"):
        print(chunk, end="", flush=True)
# 运行异步函数
import asyncio
# asyncio.run(stream_response())
await stream_response()

有一天，一只小狗走进了一家酒吧。  

它跳到吧台上，对酒保说：“给我来一杯柠檬汽水。”  

酒保很惊讶：“哇！会说话的狗！这太神奇了！”  

小狗耸耸肩：“是啊，而且我还会弹钢琴呢。”  

酒保更震惊了：“真的吗？那你能表演一下吗？”  

小狗摇摇头：“不行，我是只小短腿，够不着琴键。”

#### 短期记忆

In [41]:
# 创建虚拟工具来满足 API 要求
@tool
def dummy_tool():
    """一个虚拟工具，用于满足 DeepSeek API 工具调用的要求"""
    return "这是一个占位工具"

# 创建短期记忆
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 创建 Prompt 模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember the conversation history and context."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 创建 Agent 和 AgentExecutor
agent = create_tool_calling_agent(llm=llm, tools=[dummy_tool], prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=[dummy_tool], 
    memory=memory, 
    verbose=True,
    handle_parsing_errors=True  # 添加错误处理
)

# 测试对话
print("=== Agent 短期记忆对话测试 ===")

try:
    # 第一轮对话
    print("\n用户: 你好，我叫小明")
    response1 = agent_executor.invoke({"input": "你好，我叫小明"})
    print(f"AI: {response1['output']}")

    # 第二轮对话
    print("\n用户: 我刚才说了什么？")
    response2 = agent_executor.invoke({"input": "我刚才说了什么？"})
    print(f"AI: {response2['output']}")

    # 第三轮对话
    print("\n用户: 我的名字是什么？")
    response3 = agent_executor.invoke({"input": "我的名字是什么？"})
    print(f"AI: {response3['output']}")

except Exception as e:
    print(f"发生错误: {e}")

# 查看当前记忆内容
print("\n=== 当前记忆内容 ===")
print(memory.chat_memory.messages)

=== Agent 短期记忆对话测试 ===

用户: 你好，我叫小明


> Entering new AgentExecutor chain...
你好，小明！很高兴认识你。有什么我可以帮你的吗？

> Finished chain.
AI: 你好，小明！很高兴认识你。有什么我可以帮你的吗？

用户: 我刚才说了什么？


> Entering new AgentExecutor chain...
你刚才说：“你好，我叫小明。”  

需要我帮你做些什么吗？

> Finished chain.
AI: 你刚才说：“你好，我叫小明。”  

需要我帮你做些什么吗？

用户: 我的名字是什么？


> Entering new AgentExecutor chain...
你的名字是 **小明**。需要我帮你记住其他信息吗？

> Finished chain.
AI: 你的名字是 **小明**。需要我帮你记住其他信息吗？

=== 当前记忆内容 ===
[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，小明！很高兴认识你。有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚才说了什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚才说：“你好，我叫小明。”  \n\n需要我帮你做些什么吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的名字是什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你的名字是 **小明**。需要我帮你记住其他信息吗？', additional_kwargs={}, response_metadata={})]
